In [1]:
import pandas as pd

submission = pd.read_excel('submission.xlsx')
print(submission.shape)
submission.head()

(57388, 17)


,id,lang,create_time,judge_end_time,status,status_percent,status_detail,status_time,author_id,problem_id,status_message,code_length,judge_server,status_test,contest_time,status_memory,visible
0,2864861,c,2022-02-20 21:46:05,2022-02-20 21:46:07,-1,0.0,"[{""memory"": 0.48, ""time"": 0.005, ""verdict"": -1...",0.005,102113,1147,NaN,667,38,1,NaN,0.480,1
1,2864868,c,2022-02-20 22:07:02,2022-02-20 22:07:03,0,100.0,"[{""memory"": 0.48, ""time"": 0.005, ""verdict"": 0,...",0.005,102113,1147,NaN,797,38,0,NaN,0.480,1
2,2864879,c,2022-02-20 22:46:54,2022-02-20 22:46:55,0,100.0,"[{""memory"": 0.48, ""time"": 0.006, ""verdict"": 0,...",0.006,102113,2876,NaN,1003,38,0,NaN,0.488,1
3,2864910,c,2022-02-21 09:57:40,2022-02-21 09:57:40,0,100.0,"[{""memory"": 0.48, ""time"": 0.005, ""verdict"": 0,...",0.005,101615,3190,NaN,512,38,0,NaN,0.480,1
4,2864918,c,2022-02-21 10:49:31,2022-02-21 10:49:32,-1,0.0,"[{""memory"": 0.477, ""time"": 0.006, ""verdict"": -...",0.006,93099,3040,NaN,448,38,1,NaN,0.480,0


In [2]:
submission = submission.loc[ (submission['status'] != 6), ['author_id', 'problem_id', 'status_percent', 'id']]
submission = submission.rename(columns={'id':'code_id'})
print(submission.shape)
submission.head()

(51887, 4)


,author_id,problem_id,status_percent,code_id
0,102113,1147,0.0,2864861
1,102113,1147,100.0,2864868
2,102113,2876,100.0,2864879
3,101615,3190,100.0,2864910
4,93099,3040,0.0,2864918


In [3]:
submission.isnull().sum()
submission.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
print(submission.shape)
submission.head()

(51887, 4)


,author_id,problem_id,status_percent,code_id
0,102113,1147,0.0,2864861
1,102113,1147,100.0,2864868
2,102113,2876,100.0,2864879
3,101615,3190,100.0,2864910
4,93099,3040,0.0,2864918


In [4]:
problem = pd.read_csv('problem.csv')
problem = problem[['1', '0']]
problem = problem.values
problem.tolist()
prob2 = dict(zip(problem[:,0],problem[:,1]))
print('problem num: ',len(prob2))

problem num:  141


In [5]:
def search_prob2(x):
    tmp = prob2.get(x, 1000) - 1000
    if tmp == 0:
        print('=========')
    return tmp

submission['problem_id'] = submission['problem_id'].apply(search_prob2)

print(submission.shape)
submission.head()

(51887, 4)


,author_id,problem_id,status_percent,code_id
0,102113,1,0.0,2864861
1,102113,1,100.0,2864868
2,102113,8,100.0,2864879
3,101615,6,100.0,2864910
4,93099,7,0.0,2864918


In [6]:
author2 = dict()
cnt = 0

def search_author2(x):
    global author2
    global cnt
    tmp = author2.get(x)
    if tmp == None:
        cnt = cnt + 1
        author2[x] = cnt
        tmp = author2[x]
    return tmp

submission['author_id'] = submission['author_id'].apply(search_author2)

print('student num: ',cnt)
submission.head()

student num:  195


,author_id,problem_id,status_percent,code_id
0,1,1,0.0,2864861
1,1,1,100.0,2864868
2,1,8,100.0,2864879
3,2,6,100.0,2864910
4,3,7,0.0,2864918


In [7]:
label = pd.read_excel('EOJ.xlsx')
tmp = [label.loc[i,r'标签（整合）'].split(', ') for i in range(len(label))]
concept = set(sum(tmp,[]))
concept = list(concept)
concept.sort()
print(len(concept))
label['concept'] = 0
print(concept)
for i in range(len(label)):
    tmp = label.loc[i,r'标签（整合）'].split(', ')
    num = 0
    for index,j in enumerate(concept):
        if j in tmp:
            num = num + 2**index
    label.loc[i,'concept'] = num
label = label[['#', 'concept']]
label.loc[:,'#'] = [label.loc[i,'#']-1000 for i in range(len(label))]
label = label.rename(columns={'#':'problem_id'})
label.head()

25
['STL', 'big int', 'binary search', 'bitmasks', 'brute force', 'combinatorics', 'contruction', 'dfs', 'dp', 'dsu', 'gcd', 'graph', 'greedy', 'hash', 'implementation', 'math', 'matrix', 'parser', 'positional notation', 'prefix sum', 'recursion', 'sortings', 'stack', 'string', 'two pointer']


,problem_id,concept
0,1,278528
1,2,266240
2,3,2359296
3,4,262152
4,5,278528


In [8]:
# writer = pd.ExcelWriter(r'data.xlsx')
# submission.to_excel(writer,"submission")
# label.to_excel(writer,"Q_matrix",index=False)
# writer.save()
submission.to_csv('codes.csv', index=False)
label.to_csv('problem_label.csv', index=False)